In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# https://pythondata.com/market-basket-analysis-with-python-and-pandas/

In [2]:
df = pd.read_excel('Online_Retail.xlsx')

In [3]:
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [4]:
df[df.InvoiceNo.str.contains('C', na=False)].head()

InvoiceNo StockCode                       Description  Quantity  \
141   C536379         D                          Discount        -1   
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

            InvoiceDate  UnitPrice  CustomerID         Country  
141 2010-12-01 09:41:00      27.50     14527.0  United Kingdom  
154 2010-12-01 09:49:00       4.65     15311.0  United Kingdom  
235 2010-12-01 10:24:00       1.65     17548.0  United Kingdom  
236 2010-12-01 10:24:00       0.29     17548.0  United Kingdom  
237 2010-12-01 10:24:00       0.29     17548.0  United Kingdom

In [5]:
df = df[~df['InvoiceNo'].str.contains('C')]

In [8]:
market_basket = df[df['Country'] == 'United Kingdom'].groupby(['InvoiceNo', 'Description'])['Quantity']

In [9]:
market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [10]:
market_basket.head()

Description  20713   4 PURPLE FLOCK DINNER CANDLES  \
InvoiceNo                                            
536365         0.0                             0.0   
536366         0.0                             0.0   
536367         0.0                             0.0   
536368         0.0                             0.0   
536369         0.0                             0.0   

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
InvoiceNo                                                         
536365                                  0.0                 0.0   
536366                                  0.0                 0.0   
536367                                  0.0                 0.0   
536368                                  0.0                 0.0   
536369                                  0.0                 0.0   

Description   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY  \
InvoiceNo                                                             
536365                                0.0                       0.0   
536366                                0.0                       0.0   
536367                                0.0                       0.0   
536368                                0.0                       0.0   
536369                                0.0                       0.0   

Description   OVAL WALL MIRROR DIAMANTE    RED SPOT GIFT BAG LARGE  \
InvoiceNo                                                            
536365                               0.0                       0.0   
536366                               0.0                       0.0   
536367                               0.0                       0.0   
536368                               0.0                       0.0   
536369                               0.0                       0.0   

Description   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET  ...  \
InvoiceNo                                                               ...   
536365                                    0.0                      0.0  ...   
536366                                    0.0                      0.0  ...   
536367                                    0.0                      0.0  ...   
536368                                    0.0                      0.0  ...   
536369                                    0.0                      0.0  ...   

Description  wrongly coded 20713  wrongly coded 23343  wrongly coded-23343  \
InvoiceNo                                                                    
536365                       0.0                  0.0                  0.0   
536366                       0.0                  0.0                  0.0   
536367                       0.0                  0.0                  0.0   
536368                       0.0                  0.0                  0.0   
536369                       0.0                  0.0                  0.0   

Description  wrongly marked  wrongly marked 23343  \
InvoiceNo                                           
536365                  0.0                   0.0   
536366                  0.0                   0.0   
536367                  0.0                   0.0   
536368                  0.0                   0.0   
536369                  0.0                   0.0   

Description  wrongly marked carton 22804  wrongly marked. 23343 in box  \
InvoiceNo                                                                
536365                               0.0                           0.0   
536366                               0.0                           0.0   
536367                               0.0                           0.0   
536368                               0.0                           0.0   
536369                               0.0                           0.0   

Description  wrongly sold (22719) barcode  wrongly sold as sets  \
InvoiceNo                                                         
536365                              

In [11]:
def encode_data(datapoint):
    if datapoint <= 0:
        return 0
    if datapoint >= 1:
        return 1

In [12]:
market_basket = market_basket.applymap(encode_data) # long one

Now, lets find out which items are frequently purchased together. We do this by applying the mlxtend `apriori` fuuinction to our dataset.

In [13]:
itemsets = apriori(market_basket, min_support=0.03, use_colnames=True)

In [14]:
rules = association_rules(itemsets, metric="lift", min_threshold=0.5)

In [15]:
market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
# https://github.com/pandas-dev/pandas/issues/26314 error explained

ValueError: index must be a MultiIndex to unstack, <class 'pandas.core.indexes.base.Index'> was passed

In [16]:
from itertools import combinations, groupby
from collections import Counter

In [17]:
df_manual = df[df['Country'] =="United Kingdom"]

In [18]:
orders = df_manual.set_index('InvoiceNo')['StockCode']

In [19]:
statistics = orders.value_counts().to_frame("frequency")
statistics['support']  = statistics / len(set(orders.index)) * 100

In [20]:
min_support=0.03 # same value we used above.

items_above_support = statistics[statistics['support'] >= min_support].index
orders_above_support = orders[orders.isin(items_above_support)]

In [21]:
order_counts = orders.index.value_counts()
orders_over_two_index = order_counts[order_counts>=2].index
orders_over_two = orders[orders.index.isin(orders_over_two_index)]

In [22]:
statistics = orders_over_two.value_counts().to_frame("frequency")
statistics['support']  = statistics / len(set(orders_over_two.index))

In [23]:
def itemset_generator(orders):
    orders = orders.reset_index().values
    for order_id, order_object in groupby(orders, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
        for item_pair in combinations(item_list, 2):
            yield item_pair

itemsets_gen = itemset_generator(orders_over_two)
itemsets  = pd.Series(Counter(itemsets_gen)).to_frame("frequencyAC")
itemsets['supportAC'] = itemsets['frequencyAC'] / len(orders_over_two_index) * 100
itemsets = itemsets[itemsets['supportAC'] >= min_support]

In [24]:
# Create table of association rules and compute relevant metrics
itemsets = itemsets.reset_index().rename(columns={'level_0': 'antecedents', 'level_1': 'consequents'})

itemsets = (itemsets
     .merge(statistics.rename(columns={'freq': 'freqA', 'support': 'antecedent support'}), left_on='antecedents', right_index=True)
     .merge(statistics.rename(columns={'freq': 'freqC', 'support': 'consequents support'}), left_on='consequents', right_index=True))


itemsets['confidenceAtoC'] = itemsets['supportAC'] / itemsets['antecedent support']
itemsets['confidenceCtoA'] = itemsets['supportAC'] / itemsets['consequents support']
itemsets['lift'] = itemsets['supportAC'] / (itemsets['antecedent support'] * itemsets['consequents support'])

itemsets=itemsets[['antecedents', 'consequents','antecedent support', 'consequents support', 'confidenceAtoC','lift']]

In [25]:
rules = itemsets
rules_over_50 = rules[(rules.confidenceAtoC >0.50)]
rules_over_50.set_index('antecedents',inplace=True)
rules_over_50.reset_index(inplace=True)
rules_over_50=rules_over_50.sort_values('lift', ascending=False)

In [26]:
rules_over_50.head()

antecedents consequents  antecedent support  consequents support  \
1593850      48173c      85099f            0.000423             0.000604   
1677602      90082D      90082B            0.000363             0.000725   
1678938       20698       20697            0.000544             0.000785   
1678898      37489B      37489C            0.000725             0.000483   
649046       90129D      90129E            0.000423             0.000846   

         confidenceAtoC           lift  
1593850       71.428571  118214.285714  
1677602       83.333333  114930.555556  
1678938       88.888889  113162.393162  
1678898       50.000000  103437.500000  
649046        85.714286  101326.530612